In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.8 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://developer.download.nvi

bash: line 6: fg: no job control
bash: line 8: fg: no job control
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 5.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0   482k      0  0:00:02  0:00:02 --:--:--  932k
./configure: line 7378: /usr/bin/file: No such file or directory
In file included from viterbi.cpp:14:0:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source

In [ ]:
!pip install soynlp emoji

     |████████████████████████████████| 416 kB 4.3 MB/s 
     |████████████████████████████████| 175 kB 42.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=f908aba4ac8c9e8db988d2f2aae68306dc1c6a4732932b2a92664dc0e82b7293
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import emoji
from google.colab import files
from konlpy.tag import Okt, Mecab
from soynlp.normalizer import repeat_normalize
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('.xlsx', sheet_name = 0)
df

In [ ]:
df1 = df.drop_duplicates()
df1 = df1.dropna()
df1
# 중복값과 결측값은 각각 46(0.0288%), 12개(0.0075%)로 약 160,000개의 데이터에 굉장히 적은 양입니다. 제거해주도록 하겠습니다.

In [ ]:
df1.type.value_counts()

In [ ]:
df2 = df2.loc[~(df2[''].str.contains('', na=True) | df2[''].str.contains('', na=True))]
df2

In [ ]:
# def regex(text):
#   regex = '[^ ㄱ-ㅎㅏ-ㅣ가-힣]+'
#   subst = ''  
#   result = re.sub(regex, subst, text)
#   return result

In [ ]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [ ]:
df2['ex1'] = df2[''].apply(str).apply(clean)
for i in df2.loc[df2['ex1'] == ''].message.unique():
  print(i)
print(df2.loc[df2['ex1'] == ''].message.nunique())

In [ ]:
df3 = df2.loc[df2['ex1'] != '']
df3['ex2'] = df3['ex1'].str.replace('^ +', '')
df3['ex2'].replace('', np.nan, inplace = True)
df3 = df3.dropna().reset_index()
df3

In [ ]:
okt = Okt()
#mecab = Mecab()

df3['tokenized_pos'] = None

for idx, sent in enumerate(df3['ex2']):

  #normalized = okt.normalize(sent)
  tokenized = okt.pos(sent, norm=True, stem=True)
  
  df3['tokenized_pos'][idx] = tokenized

df_tokenized = df3[['',	'',	'', '', '', '', '']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
df_tokenized = df3[['',	'',	'', '', '', '', '']]
df_tokenized

https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0  

In [ ]:
df_tokenized['del_stopwords'] = None

list_temp = []
for idx, tokens in enumerate(df_tokenized.tokenized_pos):
  for token in tokens:
      list_temp.append(token[1])

list_temp = set(list_temp)
list_temp = list(list_temp)
list_temp

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


['Eomi',
 'Verb',
 'Adverb',
 'VerbPrefix',
 'Exclamation',
 'Conjunction',
 'Adjective',
 'PreEomi',
 'KoreanParticle',
 'Josa',
 'Noun',
 'Suffix',
 'Determiner',
 'Modifier']

In [ ]:
# df_tokenized['del_stopwords'] = None

# for idx, tokens in enumerate(df_tokenized.tokenized_pos):
#   del_stopwords_list = []
  
#   for token in tokens:
#     for j in ['VerbPrefix', 'Verb', 'Adverb', 'Adjective', 'Noun', 'Exclamation', 'KoreanParticle']:
#       if token[1] == j:
#         del_stopwords_list.append(token)
  
#   df_tokenized['del_stopwords'][idx] = del_stopwords_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# df_tokenized['del_stopwords'] = None

# for idx, tokens in enumerate(df_tokenized.tokenized_pos):
#   del_stopwords_list = []
  
#   for token in tokens:
#     for j in ['NNG', 'NNP', 'VCP', 'VCN', 'IC', 'MAG', 'VA', 'VV', 'XR']:
#       if j in token[1]:
#         del_stopwords_list.append(token)
  
#   df_tokenized['del_stopwords'][idx] = del_stopwords_list

In [ ]:
# 런타임 재실행시 데이터를 tuple, list로 인식하지않고 한 글자씩 인식하는 문제가 발생. 이 문제를 해결하기 위한 코드
# import ast
# df.tokenized_pos = df.tokenized_pos.apply(ast.literal_eval)

words_count = Counter()
df_tokenized['del_stopwords'].apply(lambda x: words_count.update(x))
words_count.most_common(20), len(words_count)

In [ ]:
temp = pd.DataFrame(df_tokenized[['', '',	'', '',	'',	'']])
temp.to_csv('DB_part2.csv', encoding = 'utf-8-sig', index = False)
files.download('DB_part2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
mecab = Mecab()
df3['nouns'] = None

for idx, sent in enumerate(df3['ex2']):
  tokenized = mecab.pos(sent)
  nouns = [s for s, t in tokenized if t in ['NNG', 'NNP'] and len(s) > 1]
  nouns = [s for s in nouns if not s in stopwords]
  df3['nouns'][idx] = nouns

df3

In [ ]:
for_lda = df3.loc[(df3.nouns.astype(str) != '[]')]
text_list = list(for_lda.nouns)

In [ ]:
words_count2 = Counter()
for_lda['nouns'].apply(lambda x: words_count2.update(x))
words_count2.most_common(), len(words_count2)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
  coherence_values = []
  model_list = []

  for num_topics in range(start, limit, step):
    model = LdaModel(corpus = corpus, id2word=dictionary, num_topics=num_topics)
    model_list.append(model)
    coherencemodel = CoherenceModel(model = model, texts = texts, dictionary = dictionary, coherence = 'c_v')
    coherence_values.append(coherencemodel.get_coherence())
  
  return model_list, coherence_values

def find_optimal_number_of_topics(dictionary, corpus, processed_data):
  limit = 20;
  start = 2;
  step = 1;

  model_list, coherence_values = compute_coherence_values(dictionary = dictionary, corpus = corpus, texts = text_list, start = start, limit = limit, step = step)

  x = range(start, limit, step)
  plt.plot(x, coherence_values)
  plt.xlabel("Num Topics")
  plt.ylabel("Coherence score")
  plt.legend(("coherence_values"), loc='best')
  plt.show()

In [ ]:
dictionary = corpora.Dictionary(text_list)
dictionary.filter_extremes(no_below=10, no_above=0.05)

corpus = [dictionary.doc2bow(text) for text in text_list]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 2569
Number of documents: 89448


In [ ]:
find_optimal_number_of_topics(dictionary, corpus, df3.nouns)

In [ ]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)

        for j, (topic_num, prop_topic) in enumerate(doc):
            if j == 0: 
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
  
            else:
                break
    return(topic_table)

In [ ]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

In [ ]:
topictable['가장 비중이 높은 토픽'] = topictable['가장 비중이 높은 토픽'].astype(int)

In [ ]:
topictable.to_csv('DB_lda2.csv', encoding = 'utf-8-sig', index = False)
files.download('DB_lda2.csv') 

In [ ]:
# okt = Okt()

# df3['normalized'] = None

# fo idx, sent in enumerate(df3['ex2']):

#   normalized = okt.normalize(sent)
  
#   df3['normalized'][idx] = normalized

#   print(idx+1, '번 째 ...')r

In [ ]:
# df_normalized = df3[['index',	'channel_url',	'sender_nickname', 'message', 'created_at', 'ex2', 'normalized']]

# df_normalized = pd.DataFrame(df_normalized)
# df_normalized.to_csv('df_normalized.csv', sep=',', na_rep='NaN', encoding = 'utf-8')

# files.download('df_normalized.csv')
# df_normalized

In [ ]:
# df3['tokenized'] = None
# df3['tokenized_pos'] = None
# df3['tokenized_noun'] = None

# for idx, sent in enumerate(df3['ex2']):

#   normalized = okt.normalize(sent)

#   tokenized1 = okt.morphs(normalized)
#   tokenized2 = okt.pos(normalized)
#   tokenized3 = okt.nouns(normalized)
  
#   df3['tokenized'][idx] = tokenized1
#   df3['tokenized_pos'][idx] = tokenized2
#   df3['tokenized_noun'][idx] = tokenized3

#   print(idx+1, '번 째 ...')

# df_tokenized = df3[['index',	'channel_url',	'sender_nickname', 'message', 'created_at', 'tokenized' , 'tokenized_pos', 'tokenized_noun']]

In [ ]:
# try:
#   for i in range(0, len(text1)):
#     text2.append(spell_checker.check(text1[i]).checked)
#     print(i+1, '번째 맞춤법 검사 ...')
# except:
#   df_checked_spell = pd.DataFrame(text2)
#   df_checked_spell.to_csv('temp.csv', sep=',', na_rep='NaN', encoding = 'utf-8')
#   files.download('temp.csv')